# Marketing Analytics Jumpstart Quick Installation


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/marketing-analytics-jumpstart/blob/main/notebooks/quick_installation.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fmarketing-analytics-jumpstart%2Fmain%2Fnotebooks%2quick_installation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/marketing-analytics-jumpstart/blob/main/notebooks/quick_installation.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/marketing-analytics-jumpstart/main/notebooks/quick_installation.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

Follow this Colab notebook to quick install the Marketing Analytics Jumpstart solution on a Google Cloud Project.

In [2]:
# @title Input Google Cloud Project ID
# prompt: set PROJECT_ID env variable and run gcloud set project

GOOGLE_CLOUD_PROJECT = "marketing-data-engine-demo" #@param {type:"string"}
GOOGLE_CLOUD_QUOTA_PROJECT = GOOGLE_CLOUD_PROJECT
PROJECT_ID = GOOGLE_CLOUD_PROJECT
!gcloud config set disable_prompts true
!gcloud config set project {PROJECT_ID}

Updated property [core/disable_prompts].
Updated property [core/project].


In [1]:
# @title Authenticate to Google Cloud Platform
# prompt: authenticate to google cloud project
from google.colab import auth
auth.authenticate_user()

In [7]:
# @title Authenticate using application default credentials Google Cloud Platform
!gcloud config set disable_prompts false
!gcloud auth application-default login --quiet --scopes="openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/sqlservice.login,https://www.googleapis.com/auth/analytics,https://www.googleapis.com/auth/analytics.edit,https://www.googleapis.com/auth/analytics.provision,https://www.googleapis.com/auth/analytics.readonly,https://www.googleapis.com/auth/accounts.reauth"
!gcloud auth application-default set-quota-project {PROJECT_ID}
!export GOOGLE_APPLICATION_CREDENTIALS=/content/.config/application_default_credentials.json

Updated property [core/disable_prompts].
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.edit+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.provision+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=upHeK5pdqi2RFOTouPRbKKe64tObPX&prompt=consent&token_usage=remote&access_type=offline&code_challenge=PKVsYphOmWNHo5bF55Usd3qUN7QB6NFHKky2Satjags&code_challenge_method=S256

In [8]:
# prompt: git clone a repository and setting cd to it
REPO="marketing-analytics-jumpstart"
!if [ ! -d "/content/{REPO}" ]; then git clone https://github.com/GoogleCloudPlatform/{REPO}.git ; fi
SOURCE_ROOT="/content/"+REPO
%cd {SOURCE_ROOT}

Cloning into 'marketing-analytics-jumpstart'...
remote: Enumerating objects: 2535, done.
remote: Counting objects: 100% (1609/1609), done.
remote: Compressing objects: 100% (846/846), done.
remote: Total 2535 (delta 1133), reused 974 (delta 756), pack-reused 926 (from 1)
Receiving objects: 100% (2535/2535), 18.58 MiB | 21.67 MiB/s, done.
Resolving deltas: 100% (1509/1509), done.
/content/marketing-analytics-jumpstart


In [9]:
%%bash
# prompt: install packages
apt-get install python3.10
CLOUDSDK_PYTHON=python3.10

#pip3 install poetry
sudo apt update
sudo apt install pipx
pipx ensurepath
pipx install poetry

export PATH="/root/.local/bin:$PATH"
poetry env use python3.10
poetry --version

git clone --depth=1 https://github.com/tfutils/tfenv.git ~/.tfenv
echo 'export PATH="~/.tfenv/bin:$PATH"' >> ~/.bash_profile
echo 'export PATH=$PATH:~/.tfenv/bin' >> ~/.bashrc
export PATH="$PATH:~/.tfenv/bin"

mkdir -p ~/.local/bin/
. ~/.profile
ln -s ~/.tfenv/bin/* ~/.local/bin
which tfenv
tfenv --version

tfenv install 1.5.7
tfenv use 1.5.7
terraform --version

export PATH="$PATH:~/.tfenv/bin"
export PROJECT_ID=$(gcloud config get project --format=json | tr -d '"')
source ./scripts/generate-tf-backend.sh

Reading package lists...
Building dependency tree...
Reading state information...
python3.10 is already the newest version (3.10.12-1~22.04.6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 34.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
creating virtual environment...
creating shared libraries...
upgrading shared libraries...
installing poetry...
⚠️  Note: '/root/.local/bin' is not on your PATH environment variable. These apps will not be
    globally accessible until your PATH is updated. Run `pipx ensurepath` to automatically add it,
    or manually modify your PATH in your shell's con

In [10]:
%%bash
TERRAFORM_RUN_DIR=$(pwd)/infrastructure/terraform
cp $TERRAFORM_RUN_DIR/terraform-sample.tfvars $TERRAFORM_RUN_DIR/terraform.tfvars -v

'/content/marketing-analytics-jumpstart/infrastructure/terraform/terraform-sample.tfvars' -> '/content/marketing-analytics-jumpstart/infrastructure/terraform/terraform.tfvars'


In [11]:
%%bash
export PATH="$PATH:~/.tfenv/bin"
export GOOGLE_APPLICATION_CREDENTIALS=/content/.config/application_default_credentials.json
TERRAFORM_RUN_DIR=$(pwd)/infrastructure/terraform
terraform -chdir="${TERRAFORM_RUN_DIR}" init


Initializing the backend...

Successfully configured the backend "gcs"! Terraform will automatically
use this backend unless the backend configuration changes.
Initializing modules...
- activation in modules/activation
- activation.activation_pipeline_container in .terraform/modules/activation.activation_pipeline_container
- activation.activation_pipeline_template in .terraform/modules/activation.activation_pipeline_template
- activation.add_invoker_binding in .terraform/modules/activation.add_invoker_binding
- activation.bigquery in .terraform/modules/activation.bigquery
- activation.build_logs_bucket in .terraform/modules/activation.build_logs_bucket/modules/simple_bucket
- activation.function_bucket in .terraform/modules/activation.function_bucket/modules/simple_bucket
- activation.pipeline_bucket in .terraform/modules/activation.pipeline_bucket/modules/simple_bucket
- activation.pipeline_service_account in .terraform/modules/activation.pipeline_service_account
- activation.project

In [16]:
%%bash
export PATH="$PATH:~/.tfenv/bin"
export PATH="/root/.local/bin:$PATH"
export GOOGLE_APPLICATION_CREDENTIALS=/content/.config/application_default_credentials.json
TERRAFORM_RUN_DIR=$(pwd)/infrastructure/terraform
terraform -chdir="${TERRAFORM_RUN_DIR}" apply -auto-approve

module.activation[0].module.add_invoker_binding.data.external.env_override[0]: Reading...
module.activation[0].module.activation_pipeline_container.data.external.env_override[0]: Reading...
module.activation[0].data.template_file.churn_propensity_query_template_file: Reading...
module.activation[0].module.activation_pipeline_template.data.external.env_override[0]: Reading...
module.monitoring[0].data.template_file.resource_link_content: Reading...
module.monitoring[0].data.template_file.resource_link_content: Read complete after 0s [id=64ce73b4b604f03edb5ec083e09452c3331c96b067b1c8b76407addb7db7f9ea]
module.activation[0].data.template_file.auto_audience_segmentation_query_template_file: Reading...
module.activation[0].data.template_file.purchase_propensity_query_template_file: Reading...
module.activation[0].data.template_file.churn_propensity_query_template_file: Read complete after 0s [id=724c878c8d7433cb69440ec7d0072b22c024049e32965018901526063e313d8e]
module.activation[0].data.temp